In [1]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 1.53 ms


In [2]:
try:
    import deepface
except:
    !pip install deepface
    import deepface

time: 2.43 ms


In [3]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageStat
import time
from deepface import DeepFace
import warnings
warnings.filterwarnings("ignore")

colab = True
tt = time.time
st = tt()

time: 1.42 s


In [4]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.24 ms


In [5]:
# for colab
colab = True
base_dir = "/content/drive/My Drive/nptel_face_recognition"
print(f"os.getcwd(): {os.getcwd()}")
if not os.getcwd() == base_dir:
    os.chdir(base_dir)
    print(f"os.getcwd(): {os.getcwd()}")

os.getcwd(): /content
os.getcwd(): /content/drive/My Drive/nptel_face_recognition
time: 4.29 ms


In [6]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

save_pickle = False
sound = True

time: 36.3 ms


In [7]:
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

known_faces_dir = os.path.join(data_dir, 'known_faces')
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('/')[-1])] for w in images_dir_list]

# known_face_images_nested_list = []
# for image_dir in images_dir_list:
#     known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
#     known_face_images_nested_list.append([face_recognition.load_image_file(w) for w in known_images_path_list])

if save_pickle:
    save_as_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"), known_face_names)
    save_as_pickle(os.path.join(pkl_files_dir, "known_face_images_nested_list.pkl"), known_face_encodings)
    save_as_pickle(os.path.join(pkl_files_dir, "known_face_seprate_encodings.pkl"), known_face_seprate_encodings)

# winsound.Beep(freq, duration)

time: 30.1 ms


In [ ]:
tracebacks_dir = os.path.join(data_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

pkl_files_dir = os.path.join(data_dir, "pkl_files")
nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

known_face_names = load_from_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"))
known_face_encodings = load_from_pickle(os.path.join(pkl_files_dir, "known_face_encodings.pkl"))

backends = ['opencv', 'ssd', 'dlib', 'mtcnn']
# backends = ['opencv', 'ssd', 'dlib']

# start after labeled faces
vedio_start_idx = 2
# treshold for face distance
max_pred_cnt = 20

num_channels = 5
num_playlists = 5
num_vedios = 5
    
# num_channels = 5
# num_playlists = 5
# num_vedios = 5

winner_backend=[]
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)

    results_chanel_dir = os.path.join(results_dir, nptel_channel_name)
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
    paylist_name_list = [x.split('/')[-1] for x in paylist_path_list] 
    
    vedio_ext = '.mp4'
    
    namef=0
    namenf=0
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
    for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
        print(f"\n\nplaylist: {playlistn}")
        if playlistn not in playlist_2_prof_name_dict.keys():
            print(f"{playlistn} not in playlist_2_prof_name_dict")
            continue
        
        true_name = playlist_2_prof_name_dict[playlistn]
        if true_name is None:
            print(f"no name for {playlistn}")
            continue
            
        if true_name not in folder_name_2_prof_name_dict.values():
            print(f"no face for {true_name}")
            continue
        
        results_playlist_dir = os.path.join(results_chanel_dir, playlistn)
        
        vedio_path_list = get_extension_files(playlist, ext='.mp4')
        vedio_name_list = [x.split('/')[-1] for x in vedio_path_list] 
        
        for vedio_path in vedio_path_list[vedio_start_idx:vedio_start_idx+num_vedios]:
            vname = vedio_path.split('/')[-1]

            video_capture = cv2.VideoCapture(vedio_path)

            vst = 0*60
            vet = 5*60
            num_frames_per_sec = 1
            pred_cnt = 0
            names_list = []

            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                 # Grab a single frame of video
                ret, frame = video_capture.read()

                # process
                if i%int(fps/num_frames_per_sec)==0:
                    
                    atleastone = False
                    for backend in backends:
                        try:
                            # TODO: PLS find a workaround this is too stupid
                            cv2.imwrite("img.jpg", frame)
                            detected_face = DeepFace.detectFace("img.jpg", detector_backend = backend)
                            names_list.append(backend)
                            atleastone = True
                        except ValueError:
                            pass
                    
                    if atleastone:
                        pred_cnt+=1
                        if pred_cnt>=max_pred_cnt:
                            break
                    
                # Hit 'q' on the keyboard to quit!
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release handle to the webcam
            video_capture.release()
            cv2.destroyAllWindows()
            
            print(f"\nvedio: {vname}")
            print(f"\ntime: {time.time()-st}")
            print("predicted face counts:")
            unique, counts = np.unique(np.array(names_list), return_counts=True)
            print(np.asarray((unique, counts)).T)
            
            max_backend = max(names_list, key=names_list.count)
            winner_backend.append(max_backend)
            max_cnt = len([w for w in names_list if w ==max_backend])
            print(f"max predictions: {max_backend} number of predictions: {max_cnt}")

print("\n\nwinner backends in all vedios:")
unique, counts = np.unique(np.array(winner_backend), return_counts=True)
print(np.asarray((unique, counts)).T)

winsound.Beep(freq, duration)




for nptel_channel_name Aerospace Engineering


playlist: Jet and Rocket Propulsion

vedio: ZDQBcuVRWYs.mp4

time: 71.0649163722992
predicted face counts:
[['dlib' '16']
 ['mtcnn' '20']
 ['opencv' '16']
 ['ssd' '16']]
max predictions: mtcnn number of predictions: 20

vedio: QLcxx6MJnbA.mp4

time: 101.56180834770203
predicted face counts:
[['dlib' '18']
 ['mtcnn' '20']
 ['opencv' '18']
 ['ssd' '10']]
max predictions: mtcnn number of predictions: 20
